In [1]:
# Importa as bibliotecas necessárias
import pandas as pd
import datetime
import re
import os
import lxml
import html5lib
from bs4 import BeautifulSoup
import requests

In [2]:
# Define a localização de arquivos e variáveis importantes
HISTORICO = '../dados/corona_brasil_old.csv'
LINK_MINSAUDE = '../dados/auxiliares/link_minsaude.csv'

meses = {
  "Janeiro": "01",
  "Janeiro": "02",
  "Março": "03",
  "Abril": "04",
  "Maio": "05",
}
HOJE = str(datetime.date.today())

HOJE

'2020-03-30'

In [4]:
# Lê a tabela com links do Ministério da Saúde e encontra o dia de hoje
acervo = pd.read_csv(LINK_MINSAUDE)

url = acervo[acervo.Data == HOJE].Link.values[0]

url

'https://www.saude.gov.br/noticias/agencia-saude/46619-brasil-registra-4-256-casos-confirmados-de-coronavirus-e-136-mortes'

In [5]:
# Baixa o link 
arquivo = requests.get(url).text

In [6]:
# Extração do dia a partir do texto da notícia
soup = BeautifulSoup(arquivo, 'lxml') 

ultima_modificacao = soup.find('span', {'class' : 'documentModified'}).text

timedate = re.search("[0-9][^\\t]*", ultima_modificacao).group().split(",")[0].split(" de ")

dia = timedate[0]
mes = meses[timedate[1]]
ano = timedate[2]

DIA = ano + '-' + mes + '-' + dia

DIA

'2020-03-29'

In [7]:
# Lê o arquivo HTML com os casos por estado

table = soup.find_all('table')[0] 

rows = table.find_all('tr')

df = []

for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    df.append([ele for ele in cols if ele])
    
df = pd.DataFrame(df)

# Define nome das colunas
df.columns = df.iloc[0]

# Retira colunas desnecessárias para a conciliação da série histórica
df = df.iloc[2:,1:4]

df

,UF/REGIÃO,CONFIRMADOS,ÓBITOS
2,227 (5%),1,"0,4%"
3,AC,34,-
4,AM,140,1
5,AP,4,-
6,PA,18,-
7,RO,6,-
8,RR,16,-
9,TO,9,-
10,720 (17%),13,"1,8%"
11,AL,17,-


In [8]:
# Remove entradas impróprias
df = df[df.CONFIRMADOS.notnull()]

# Remove valores percentuais
df = df[~df['ÓBITOS'].str.contains('%')] 
# Tira o ponto como separador decimal
df.CONFIRMADOS = df.CONFIRMADOS.str.replace('.', '') 

df

,UF/REGIÃO,CONFIRMADOS,ÓBITOS
3,AC,34,-
4,AM,140,1
5,AP,4,-
6,PA,18,-
7,RO,6,-
8,RR,16,-
9,TO,9,-
11,AL,17,-
12,BA,154,1
13,CE,348,5


In [18]:
# Adiciona coluna de data e renomeia
df['date'] = DIA

df.rename(columns={'UF/REGIÃO':'uf', 'ÓBITOS':'deaths', 'CONFIRMADOS':'cases'}, inplace=True)
df.deaths = df.deaths.str.replace('-', '0') 

df

,uf,cases,deaths,date
3,AC,34,0,2020-03-29
4,AM,140,1,2020-03-29
5,AP,4,0,2020-03-29
6,PA,18,0,2020-03-29
7,RO,6,0,2020-03-29
8,RR,16,0,2020-03-29
9,TO,9,0,2020-03-29
11,AL,17,0,2020-03-29
12,BA,154,1,2020-03-29
13,CE,348,5,2020-03-29


In [20]:
# Carrega a série histórica e junta com o dia atual, removendo duplicadas
final = pd.read_csv(HISTORICO).append(df,ignore_index=True)

# Corrige o campo de óbitos
final.deaths = final["deaths"].fillna(0) 

# TO DO: Remover entradas duplicadas com o drop_duplicates()

final.tail(100)

,date,cases,deaths,uf
866,2020-03-26,104,0,BA
867,2020-03-26,235,3,CE
868,2020-03-26,10,0,MA
869,2020-03-26,5,0,PB
870,2020-03-26,48,3,PE
...,...,...,...,...
961,2020-03-29,36,0,MS
962,2020-03-29,16,0,MT
963,2020-03-29,148,2,PR
964,2020-03-29,194,1,SC


In [23]:
# Exporta o CSV
final.deaths = final.deaths.astype(int)

final.to_csv("../dados/corona_brasil.csv", index = False)